# Plot all products

Plot all products from the cleaned 50k data set for use with the interactive heatmap

In [ ]:
import pathlib
import re
import sys

import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import rdMolDraw2D

sys.path.append(str(pathlib.Path().resolve().parents[1]))
from src.util.db_utils import SynFermDatabaseConnection

In [ ]:
# connect to DB
con = SynFermDatabaseConnection()

In [ ]:
res = con.con.execute('SELECT e.id, r.I_long, r.M_long, r.T_long, r.product_A_smiles, r.reaction_smiles, l.binary_A, l.binary_B, l.binary_C, l.binary_D, l.binary_E, l.binary_F, l.binary_G, l.binary_H, l.scaled_A, l.scaled_B, l.scaled_C, l.scaled_D, l.scaled_E, l.scaled_F, l.scaled_G, l.scaled_H, l."major_A-C" FROM experiments e LEFT JOIN representations r on e.id = r.experiment_id INNER JOIN labels l on e.id = l.experiment_id;').fetchall()

columns = [
    "experiment_id", 
    "I_long", 
    "M_long", 
    "T_long", 
    "product_A_smiles", 
    "reaction_smiles", 
    "binary_A", 
    "binary_B", 
    "binary_C", 
    "binary_D", 
    "binary_E", 
    "binary_F", 
    "binary_G", 
    "binary_H", 
    "scaled_A", 
    "scaled_B", 
    "scaled_C", 
    "scaled_D", 
    "scaled_E", 
    "scaled_F", 
    "scaled_G", 
    "scaled_H", 
    "major_A-C",
]
df = pd.DataFrame(res, columns=columns)

In [ ]:
df.head()

In [ ]:
template = Chem.RWMol(Chem.MolFromSmarts("[#6]-C(=O)-[NH1]-C(-[*])-C(-[*])-c1:n:[n,c]:c:s:1"))

AllChem.Compute2DCoords(template, canonOrient=False, sampleSeed=50, nSample=200)
template

In [ ]:
# n.b. we use the SE-ethers to get a nice orientation for the molecule
template = Chem.RWMol(Chem.MolFromSmarts("[#6]-C(=O)-[NH1]-C(-[Se]1)-C(-[Se]-C-C-1)-c1:n:[n,c]:c:s:1"))

AllChem.Compute2DCoords(template, canonOrient=True, sampleSeed=0, nSample=1)
template

In [ ]:
for atom in template.GetAtoms():
    print(atom.GetIdx(), atom.GetSymbol())

In [ ]:
# delete the placeholder atoms
template.RemoveAtom(9)
template.RemoveAtom(8)
template.RemoveAtom(7)
template.RemoveAtom(5)

In [ ]:
template

In [ ]:
for i, row in df.iterrows():
    m = Chem.MolFromSmiles(row["product_A_smiles"])
    _ = AllChem.GenerateDepictionMatching2DStructure(m, template, allowRGroups=True)

    d = rdMolDraw2D.MolDraw2DSVG(300, 300)
    d.DrawMolecule(m)
    d.FinishDrawing()
    with open(f"product_images/{row['I_long']}_{row['M_long']}_{row['T_long']}.svg", "w") as f:
        f.write(re.sub('<rect.*</rect>\n', '', d.GetDrawingText()))  # remove background with regex